# Homework 6

In [70]:
import pandas as pd
import numpy as np
from numpy import exp
%precision 8

'%.8f'

In [18]:
table_A = pd.DataFrame({"num_insured":[68799, 10325, 820, 51, 5]})
table_B = pd.DataFrame({"claim_amount":[1000, 2000, 3000],
                        "prob_claim":[0.2, 0.4, 0.4]})

In [49]:
# Lets assume, N~Poisson(lambda)
table_A

,num_insured
0,68799
1,10325
2,820
3,51
4,5


In [28]:
# Expectation E[X] & Variance V(N) for the number of insured
EN = sum((table_A.index * table_A.num_insured) / sum(table_A.num_insured))
VN = sum(((table_A.index - EN)**2 * table_A.num_insured)/sum(table_A.num_insured))

In [46]:
print("E[N] ≈ {:.6f}".format(EN))
print("V[N] ≈ {:.6f}".format(VN))

E[N] ≈ 0.151725
V[N] ≈ 0.153780


**Assume $N\sim pois(\lambda)$; do a statistical test with panger approximation**

By *Panger* 
$$
    \mathbb{P}(N=n) \approx (a + \frac{b}{n})\mathbb{P}(N=n-1)
$$

where, for $N \sim poisson(\lambda)$, $a = 0$, $b=\lambda$, and 

$$
    \mathbb{P}(N=0) = e^{-\lambda}
$$

In [71]:
total_insured = sum(table_A.num_insured)

# Panger approximation
Pn = np.zeros(len(table_A))

for ix in range(len(Pn)):
    if ix == 0:
        Pn[ix] = exp(-EN)
    else:
        Pn[ix] = (EN/ix) * Pn[ix-1]

In [78]:
test_table = pd.DataFrame({"PN":Pn,
              "expected_insured": np.round(Pn * total_insured, 0),
              "observed_insured": table_A.num_insured})
test_table

,PN,expected_insured,observed_insured
0,0.859225,68738.0,68799
1,0.130366,10429.0,10325
2,0.009890,791.0,820
3,0.000500,40.0,51
4,0.000019,2.0,5


In [88]:
# If values < 75 => merge them
test_table.ix[2,:] = np.sum(test_table.ix[2:,:])
test_table = test_table.ix[:2,:]
test_table

,PN,expected_insured,observed_insured
0,0.859225,68738.0,68799.0
1,0.130366,10429.0,10325.0
2,0.011447,917.0,988.0


In [93]:
sum((test_table.expected_insured - test_table.observed_insured)**2) / sum(test_table.expected_insured)

0.24446831

Our statistic does not pass the $\chi^2$ test. We reject $H_0$ and, since $\mathbb{V}(N) > \mathbb{E}(N)$, we assume that the distribution is a **negative binomial**.

For negative binomial, the MME for $r$ and $p$ are:
$$
    p = \frac{\bar{X}}{S^2}
$$

$$
    r = \frac{\bar{X}}{S^2 - \bar{X}}
$$

Furthermore, recall that 
$$
    R = \frac{2\theta\mathbb{E}(N)\mathbb{E}(X)}{\mathbb{E}[N]\mathbb{V}(X) + \mathbb{E}^2[X]\mathbb{V}(N)}
$$

In [125]:
p_nbin = EN / VN
r_nbin = EN / (VN - EN)

# Since we are assuming a negative binomial, we set the parameters to be:
EN = r_nbin * (1 - p_nbin) / p_nbin
VN = r_nbin * (1 - p_nbin) / p_nbin ** 2

In [126]:
EX = sum(table_B.claim_amount * table_B.prob_claim)
VX = sum((table_B.claim_amount - EX) ** 2 * table_B.prob_claim)

theta = 5 / 100 # fee
mu = 100000 # margin

R = lambda theta, EN, EX, VN, VX: 2 * theta * EN * EX / (EN * VX + EX**2 * VN)


epsilon = exp(-round(R(theta, EN, EX, VN, VX), 8) * mu) # Ruin probability

In [127]:
print("The ruin probability is: {:8f}".format(epsilon))

The ruin probability is: 0.017863
